# Quantification of nuclear translocation of ERK1/2

## Import libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import ndimage as ndi
from skimage import io, color, morphology, filters, morphology, measure
from skimage.filters import threshold_local, threshold_otsu
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops
from skimage.segmentation import clear_border, expand_labels, watershed
from skimage.color import rgb2gray
from skimage.morphology import flood_fill
import skimage as ski
from skimage import exposure
from skimage.morphology import erosion, disk

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

## Configuration for the analysis

In [ ]:
# Parameters
image = np.zeros((1024, 1360))
raw_data_folder = './data/raw/'
processed_data_folder = './data/processed/'
quantified_data_folder = './data/quantified/'
conditions = [ 'Ctrl', 'Go', 'Go pma', 'Go pn', 'Pma', 'Pn']
erk_channel = 1 # Green channel
nucleus_channel = 2 # Blue channel

# Image quantification parameters
region_extension_distance_1 = 1
region_extension_distance_2 = 4

# Data post-processing / filtering
nucleus_area_min = 250 
nucleus_area_max = 3000
nucleus_excentricity = 0.7
threshold = 4.0

if not os.path.exists(processed_data_folder):
    os.makedirs(processed_data_folder)

if not os.path.exists(quantified_data_folder):
    os.makedirs(quantified_data_folder)

## Methods used for the quantification

In [ ]:
def load_image_pair(condition_path, condition, image_name_ending,image_format):
    """Load image pair"""
    # print(f'{condition_path}/{condition} E{image_name_ending}.{image_format}')
    # print(f'{condition_path}/{condition} H{image_name_ending}.{image_format}')
    erk_image  = io.imread(f'{condition_path}/{condition} E{image_name_ending}.{image_format}')
    nucleus_image = io.imread(f'{condition_path}/{condition} H{image_name_ending}.{image_format}')

    return erk_image, nucleus_image

def select_image_channel(img, channel):
    """Select image channel"""
    return img[:,:,channel]

def convert_image_to_uint8(img):
    """Convert image to unit8 signal range to [0,255] -  applying linear scaling"""
    normalized_img = img / np.max(img)

    return (normalized_img * 255).astype(np.uint8)

def stretch_image_contrast(img):
    """Stretching image contrast"""
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))

    return img_rescale

def preprocess_image_pair(erk_img, nucleus_img):
    """Preprocess image pair"""
    # Select_channels
    if len(erk_img.shape) > 2:
        erk_img = select_image_channel(erk_img,erk_channel)
    if len(nucleus_img.shape) > 2:
        nucleus_img = select_image_channel(nucleus_img, nucleus_channel)
    
    # Convert images to unit8
    erk_channel_gs = convert_image_to_uint8(erk_img)
    nucleus_channel_gs = convert_image_to_uint8(nucleus_img)    
    
    # Stretch image contrast
    erk_channel_gs = stretch_image_contrast(erk_channel_gs)
    nucleus_channel_gs = stretch_image_contrast(nucleus_channel_gs)

    return erk_channel_gs, nucleus_channel_gs


def segment_nuclei(image):
    elevation_map = ski.filters.sobel(image)
    markers = np.zeros_like(image)
    markers[image < 40] = 1
    markers[image > 150] = 2
    segmentation = ski.segmentation.watershed(elevation_map, markers)
    bw = ndi.binary_fill_holes(segmentation - 1)
    bw = morphology.remove_small_objects(bw, 200)
    bw = clear_border(bw)
   

    distance = ndi.distance_transform_edt(bw)
    coords = peak_local_max(distance, footprint=np.ones((11, 11)), labels=bw)
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndi.label(mask)
    labels = watershed(-distance, markers, mask=bw)

    return labels

def identify_nuclei(bw):
    # Identify nucleus
    
    labeled_bw = label(bw, connectivity=2)
    regions = regionprops(labeled_bw)
    
    expanded_small = expand_labels(labeled_bw, distance=region_extension_distance_1)
    expanded_large = expand_labels(labeled_bw, distance=region_extension_distance_2)

    expanded_regions_small = regionprops(expanded_small)
    expanded_regions_large = regionprops(expanded_large)
    
    return regions, expanded_regions_small, expanded_regions_large


def segment_ekr_staining(image):
    thresh = threshold_otsu(image)
    bw = image > thresh
    bw = bw.astype(int)
    
    return bw

def create_image_from_region(region):
    image_ring = np.zeros_like(image)
    for coord in region.coords:
        image_ring[coord[0], coord[1]] = 1
    
    return image_ring

def create_nucleus_expansion_ring(region, expanded_region_small, expanded_region_large):

    # Define the structuring element (disk of radius 1)
    selem = disk(1)
    
    # Create an image from the region
    nucleus = create_image_from_region(region)
    
    # Perform erosion to shrink labeled regions
    shrunken_nucleus = erosion(nucleus, selem)

    # Create nuclear ring
    ring = create_image_from_region(expanded_region_large)-create_image_from_region(expanded_region_small)
    
    return shrunken_nucleus, ring


## Run quanfication

In [ ]:
# Name of the columns being created as part of the quantification
columns = [
    'condition_name','image_number','cell_number',
    'signal_in_nucleus', 'signal_in_ring',
    'nucleus_area','nucleus_num_pixels','nucleus_excentricity',
    'expanded_nucleus_area','expanded_nucleus_num_pixels','expanded_nucleus_excentricity',
    'number_erk_staining_nucleus_pixel', 'number_erk_staining_ring_pixel',
]

for condition in conditions:
    condition_path = raw_data_folder + '/' + condition
    contents = [c for c in os.listdir(condition_path) if c.startswith(condition)]
    erk_signal_image_names = [c for c in contents if c.startswith(f'{condition} E')]
    for erk_image_name in erk_signal_image_names:

        # Get image name parts:
        erk_image_name_parts = erk_image_name.split('.')
        image_name = erk_image_name_parts[0]
        image_format = erk_image_name_parts[1]
        image_number = image_name[-8:-6]
        image_name_ending = image_name[-8:]
        
        # Condition data array
        data_array = []
        
        # Load image pair
        erk_img, nucleus_img = load_image_pair(condition_path, condition, image_name_ending, image_format)

        # Pre-process images 
        erk_img_gs, nucleus_img_gs = preprocess_image_pair(erk_img, nucleus_img)
        
        # Segment cells using EKR staining
        erk_img_bw = segment_ekr_staining(erk_img_gs)
        
        # Segment nuclei
        nucleus_img_bw = segment_nuclei(nucleus_img_gs)
        
        # Identify nuclei
        nucleus_regions, expanded_nucleus_regions_small, expanded_nucleus_regions_large  = identify_nuclei(nucleus_img_bw)
        
        # Define signal quantities per nucleus
        b_channel = np.zeros(image.shape)
        r_channel = np.zeros(image.shape)

        # Quantify ERK signal within and around the nucleus
        for j in tqdm(range(len(nucleus_regions))):  
            nucleus, ring = create_nucleus_expansion_ring(nucleus_regions[j], expanded_nucleus_regions_small[j],expanded_nucleus_regions_large[j])
            signal_in_nucleus = np.mean(nucleus * erk_img_gs * erk_img_bw)
            signal_in_ring = np.mean(ring * erk_img_gs * erk_img_bw)
            # ratio = signal_in_nucleus / signal_in_ring

            number_erk_staining_nucleus_pixel = np.sum(np.sum(nucleus * erk_img_bw))
            number_erk_staining_ring_pixel = np.sum(np.sum(ring * erk_img_bw))

            data_array.append([
                condition,
                int(image_number),
                j,
                signal_in_nucleus,
                signal_in_ring,
                nucleus_regions[j].area,
                nucleus_regions[j].num_pixels,
                nucleus_regions[j].eccentricity,
                expanded_nucleus_regions_small[j].area,
                expanded_nucleus_regions_small[j].num_pixels,
                expanded_nucleus_regions_small[j].eccentricity,
                number_erk_staining_nucleus_pixel,
                number_erk_staining_ring_pixel,
            ])
            
            # Stack the channels along the third dimension to create an RGB image
            b_channel = b_channel + nucleus
            r_channel = r_channel + ring
            
        # Save the RGB image as a PNG file        
        if b_channel.dtype != np.uint8:
            b_channel = (b_channel * 255).astype(np.uint8)
        if r_channel.dtype != np.uint8:
            r_channel = (r_channel * 255).astype(np.uint8)
        rgb_image = np.stack((r_channel, erk_img_gs, b_channel), axis=-1)
        io.imsave(f'./data/processed/{erk_image_name[:-4]}_merged.png', rgb_image)

        # Save quantified data
        df = pd.DataFrame(data=data_array, columns=columns)
        df['ratio'] = df['signal_in_nucleus']/df['signal_in_ring']
        df.to_parquet(f'./data/processed/{image_number}.parquet',engine='pyarrow')
        


## Loading quantified data

In [ ]:
parent_folder = './data/processed/'
parquet_files = [file for file in os.listdir(parent_folder) if file.endswith('.parquet')] 
parquet_df = []
for pf in parquet_files:
    parquet_df.append(pd.read_parquet('./data/processed/'+pf))

df = pd.concat(parquet_df,axis=0)

df['condition_name'].unique()

## Post-processing of quantified data

In [ ]:
# Filter cells based on minimum, maximum nuclear area and excentricity of the nucleus
df2 = df[df['nucleus_area']>nucleus_area_min]
df2 = df2[df2['nucleus_excentricity'] < nucleus_excentricity]
df2 = df2[df2['nucleus_area'] < nucleus_area_max]

# Flag cells if the ratio in average signal is larger than the defined threshold
df2['is_positive'] = df2['ratio']>threshold
# Calculate the percentage of positive cells per experiment_name
positive_per_experiment = df2.groupby('condition_name')['is_positive'].mean() * 100

# Calculate the percentage of positive cells per experiment_name and image_number
df_result =  (
    (df2
    .groupby(['condition_name', 'image_number'])['is_positive'].mean() * 100)
    .reset_index()
)

df_cell_number_count = df2.groupby(['condition_name', 'image_number'])['cell_number'].nunique().reset_index(name='cell_count')
df_average_cell_number_per_image = df_cell_number_count.groupby(['condition_name','image_number'])['cell_count'].mean().reset_index()

# Mean & std of cell count from images per condition
df_temp1 = df_cell_number_count.groupby(['condition_name'])['cell_count'].mean().reset_index().rename(columns={'cell_count':'mean_cell_count'})
df_temp2 = df_cell_number_count.groupby('condition_name')['cell_count'].std().reset_index().rename(columns={'cell_count':'std_cell_count'})
df_temp3 = df_temp1.merge(df_temp2, on='condition_name')

## Create results csv files

In [ ]:
df_result.to_csv(f'./{quantified_data_folder}/analysis_result.csv')
df_average_cell_number_per_image.to_csv(f'./{quantified_data_folder}/cell_number_per_image.csv')
df_temp3.to_csv(f'./{quantified_data_folder}/mean_and_std_cell_number_per_condition.csv')